In [1]:
from sklearn.pipeline import make_pipeline, make_union
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from pathlib import Path

import pandas as pd

In [2]:
import sys
sys.path.append('/Users/przivic/prog/machine_learning_practico')

In [3]:
from lib import data, transformers
from lib.model import get_features_pipe, get_model_pipe

In [4]:
PATH = Path('../../data/')
movies_df = data.load_data(PATH)

Loading title basics...


/Users/przivic/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3343: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Loading title ratings...
Loading movie directors...
Merging everything...


In [5]:
principals_df = pd.read_csv(PATH / 'title.principals.tsv', sep='\t')

In [6]:
principals_df.head()

tconst  ordering     nconst         category                      job  \
0  tt0000001         1  nm1588970             self                       \N   
1  tt0000001         2  nm0005690         director                       \N   
2  tt0000001         3  nm0374658  cinematographer  director of photography   
3  tt0000002         1  nm0721526         director                       \N   
4  tt0000002         2  nm1335271         composer                       \N   

  characters  
0   ["Self"]  
1         \N  
2         \N  
3         \N  
4         \N

In [7]:
principals_df.category.value_counts()

actor                  9428011
actress                7087536
self                   6992466
writer                 5443248
director               4739306
producer               2533985
cinematographer        1472541
composer               1461940
editor                 1375324
production_designer     302528
archive_footage         245691
archive_sound             2605
Name: category, dtype: int64

In [8]:
# Sacado del codigo de directores

movies_stars = principals_df[principals_df.category.isin(['actress', 'actor'])].copy()

# Calculo un ranking por pelicula segun el ordering
movies_stars['star_rank'] = (
    movies_stars.sort_values('ordering')
        .groupby('tconst')
        .cumcount()
)

first_star = movies_stars[movies_stars.star_rank == 0][['nconst', 'tconst']].rename(columns={'nconst': '1st_star'})
second_star = movies_stars[movies_stars.star_rank == 1][['nconst', 'tconst']].rename(columns={'nconst': '2nd_star'})
third_star = movies_stars[movies_stars.star_rank == 2][['nconst', 'tconst']].rename(columns={'nconst': '3rd_star'})

In [9]:
stars_df = (
    first_star.merge(second_star, how='left', on='tconst')
              .merge(third_star, how='left', on='tconst')
)

In [10]:
stars_df.head()

1st_star     tconst   2nd_star   3rd_star
0  nm0443482  tt0000005  nm0653042        NaN
1  nm0179163  tt0000007  nm0183947        NaN
2  nm0653028  tt0000008        NaN        NaN
3  nm0063086  tt0000009  nm0183823  nm1309758
4  nm3692297  tt0000011        NaN        NaN

In [11]:
stars_df[stars_df.tconst == 'tt0120338']

1st_star     tconst   2nd_star   3rd_star
107363  nm0000138  tt0120338  nm0000701  nm0000708

In [12]:
movies_df = movies_df.merge(stars_df, on='tconst', how='left')

In [13]:
movies_df.head()

tconst titleType                    primaryTitle  \
0  tt0000009     movie                      Miss Jerry   
1  tt0000147     movie   The Corbett-Fitzsimmons Fight   
2  tt0000502     movie                        Bohemios   
3  tt0000574     movie     The Story of the Kelly Gang   
4  tt0000679     movie  The Fairylogue and Radio-Plays   

                    originalTitle  isAdult  startYear endYear  runtimeMinutes  \
0                      Miss Jerry        0     1894.0      \N            45.0   
1   The Corbett-Fitzsimmons Fight        0     1897.0      \N            20.0   
2                        Bohemios        0     1905.0      \N           100.0   
3     The Story of the Kelly Gang        0     1906.0      \N            70.0   
4  The Fairylogue and Radio-Plays        0     1908.0      \N           120.0   

                       genres  averageRating  numVotes   director   1st_star  \
0                   [Romance]            5.9       154  nm0085156  nm0063086   
1  [Documentary, News, Sport]            5.2       356  nm0714557        NaN   
2                  [no-genre]            3.8         6  nm0063413  nm0215752   
3   [Biography, Crime, Drama]            6.1       589  nm0846879  nm0846887   
4        [Adventure, Fantasy]            5.2        37  nm0091767  nm0000875   

    2nd_star   3rd_star  
0  nm0183823  nm1309758  
1        NaN        NaN  
2  nm0252720        NaN  
3  nm0846894  nm3002376  
4  nm0122665  nm0933446

# Ahora  vamos a experimentar!

Como podemos hacer para usar 1st_star y 2nd_star con el código que **ya** tenemos? [Miremos el diff](https://github.com/elsonidoq/machine_learning_practico/commit/1244da3daee2f7aff140d202885e6e8dba55c099)

In [14]:
rating_data = data.load_rating_train_dev_test(movies_df)

In [31]:
pipe = make_pipeline(
    transformers.CrewFeatures('1st_star', min_cnt_movies=3),
    DictVectorizer(sparse=False),
    StandardScaler(),
    LogisticRegression()
)

In [32]:
pipe.fit(rating_data['X_train'], rating_data['y_train'] > 7.5)

tr_auc = roc_auc_score(rating_data['y_train'] > 7.5, pipe.predict_proba(rating_data['X_train'])[:, 1])
dev_auc = roc_auc_score(rating_data['y_dev'] > 7.5, pipe.predict_proba(rating_data['X_dev'])[:, 1])

tr_auc, dev_auc

(0.6694917991958195, 0.5516081278474014)

In [33]:
pipe = make_pipeline(
    make_union(
        make_pipeline(transformers.CrewFeatures('1st_star', min_cnt_movies=3), DictVectorizer(sparse=False)),
        make_pipeline(transformers.CrewFeatures('2nd_star', min_cnt_movies=3), DictVectorizer(sparse=False)),
        make_pipeline(transformers.CrewFeatures('3rd_star', min_cnt_movies=3), DictVectorizer(sparse=False)),
    ),
    StandardScaler(),
    LogisticRegression()
)

In [34]:
pipe.fit(rating_data['X_train'], rating_data['y_train'] > 7.5)

tr_auc = roc_auc_score(rating_data['y_train'] > 7.5, pipe.predict_proba(rating_data['X_train'])[:, 1])
dev_auc = roc_auc_score(rating_data['y_dev'] > 7.5, pipe.predict_proba(rating_data['X_dev'])[:, 1])

tr_auc, dev_auc

(0.7266249341879818, 0.5782582569516197)

# Probando todo junto

In [35]:
pipe = make_pipeline(
    make_union(
        make_pipeline(transformers.CrewFeatures('1st_star', min_cnt_movies=3), DictVectorizer(sparse=False)),
        make_pipeline(transformers.CrewFeatures('2nd_star', min_cnt_movies=3), DictVectorizer(sparse=False)),
        make_pipeline(transformers.CrewFeatures('3rd_star', min_cnt_movies=3), DictVectorizer(sparse=False)),
        make_pipeline(transformers.CrewFeatures('director', min_cnt_movies=3), DictVectorizer(sparse=False)),
        make_pipeline(transformers.YearsAgo(), DictVectorizer(sparse=False)),
        make_pipeline(transformers.GenreDummies(), DictVectorizer(sparse=False)),
    ),
    StandardScaler(),
    LogisticRegression()
)

In [36]:
pipe.fit(rating_data['X_train'], rating_data['y_train'] > 7.5)

tr_auc = roc_auc_score(rating_data['y_train'] > 7.5, pipe.predict_proba(rating_data['X_train'])[:, 1])
dev_auc = roc_auc_score(rating_data['y_dev'] > 7.5, pipe.predict_proba(rating_data['X_dev'])[:, 1])

tr_auc, dev_auc

(0.8433724090930335, 0.7368580621136858)

In [37]:
pipe = make_pipeline(
    make_union(
        make_pipeline(transformers.CrewFeatures('director', min_cnt_movies=3), DictVectorizer(sparse=False)),
        make_pipeline(transformers.YearsAgo(), DictVectorizer(sparse=False)),
        make_pipeline(transformers.GenreDummies(), DictVectorizer(sparse=False)),
    ),
    StandardScaler(),
    LogisticRegression()
)

In [38]:
pipe.fit(rating_data['X_train'], rating_data['y_train'] > 7.5)

tr_auc = roc_auc_score(rating_data['y_train'] > 7.5, pipe.predict_proba(rating_data['X_train'])[:, 1])
dev_auc = roc_auc_score(rating_data['y_dev'] > 7.5, pipe.predict_proba(rating_data['X_dev'])[:, 1])

tr_auc, dev_auc

(0.8189300131210699, 0.7315217582213612)

In [39]:
pipe = make_pipeline(
    make_union(
        make_pipeline(transformers.YearsAgo(), DictVectorizer(sparse=False)),
        make_pipeline(transformers.GenreDummies(), DictVectorizer(sparse=False)),
    ),
    StandardScaler(),
    LogisticRegression()
)

In [40]:
pipe.fit(rating_data['X_train'], rating_data['y_train'] > 7.5)

tr_auc = roc_auc_score(rating_data['y_train'] > 7.5, pipe.predict_proba(rating_data['X_train'])[:, 1])
dev_auc = roc_auc_score(rating_data['y_dev'] > 7.5, pipe.predict_proba(rating_data['X_dev'])[:, 1])

tr_auc, dev_auc

(0.7419233952069703, 0.7288300431976487)

In [41]:
from sklearn.ensemble import GradientBoostingClassifier

pipe = make_pipeline(
    make_union(
        make_pipeline(transformers.CrewFeatures('1st_star', min_cnt_movies=3), DictVectorizer(sparse=False)),
        make_pipeline(transformers.CrewFeatures('2nd_star', min_cnt_movies=3), DictVectorizer(sparse=False)),
        make_pipeline(transformers.CrewFeatures('3rd_star', min_cnt_movies=3), DictVectorizer(sparse=False)),
        make_pipeline(transformers.CrewFeatures('director', min_cnt_movies=3), DictVectorizer(sparse=False)),
        make_pipeline(transformers.YearsAgo(), DictVectorizer(sparse=False)),
        make_pipeline(transformers.GenreDummies(), DictVectorizer(sparse=False)),
    ),
    GradientBoostingClassifier(),
)

In [42]:
pipe.fit(rating_data['X_train'], rating_data['y_train'] > 7.5)

tr_auc = roc_auc_score(rating_data['y_train'] > 7.5, pipe.predict_proba(rating_data['X_train'])[:, 1])
dev_auc = roc_auc_score(rating_data['y_dev'] > 7.5, pipe.predict_proba(rating_data['X_dev'])[:, 1])

tr_auc, dev_auc

(0.9175962044769267, 0.7461476500702391)

# Word2Vec features

In [43]:
class EpochSaver: pass

from gensim.models import Word2Vec

w2v = Word2Vec.load('../../data/w2v/epoch_10')

In [53]:
import numpy as np

default_vector = np.mean(w2v.wv.vectors, axis=0)

(100,)

### TODO: revisar a quien se parece este default_vector

Seria mejor un vector se 0s? alguna otra agregacion sobre los datos?

In [56]:
x_i = rating_data['X_train'][0]
fields = ['1st_star', '2nd_star', '3rd_star', 'director']
min_cnt_movies = 2

vectors = []
for field in fields:
    person_id = x_i[field]
    if person_id not in w2v.wv or w2v.wv.vocab[person_id].count < min_cnt_movies: continue
        
    vectors.append(w2v.wv[person_id])

if len(vectors) == 0:
    result = default_vector
else:
    result = np.mean(vectors, axis=0)

In [62]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

class W2VCrewFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, w2v, fields, min_cnt_movies=2):
        self.fields = fields
        self.min_cnt_movies = min_cnt_movies
        self.w2v = w2v

    def fit(self, X, y):
        self.default_vector_ = np.mean(w2v.wv.vectors, axis=0)
        return self
    
    def _get_movie_vector(self, x_i):
        vectors = []
        for field in self.fields:
            person_id = x_i[field]
            if person_id not in self.w2v.wv or self.w2v.wv.vocab[person_id].count < self.min_cnt_movies: continue

            vectors.append(self.w2v.wv[person_id])

        if len(vectors) == 0:
            return self.default_vector_
        else:
            return np.mean(vectors, axis=0)
            
    def transform(self, X):
        return np.asarray([self._get_movie_vector(x_i) for x_i in X])

In [63]:
pipe = make_pipeline(
    W2VCrewFeatures(w2v, ['1st_star', '2nd_star', '3rd_star']),
    StandardScaler(),
    LogisticRegression()
)

In [64]:
pipe.fit(rating_data['X_train'], rating_data['y_train'] > 7.5)

tr_auc = roc_auc_score(rating_data['y_train'] > 7.5, pipe.predict_proba(rating_data['X_train'])[:, 1])
dev_auc = roc_auc_score(rating_data['y_dev'] > 7.5, pipe.predict_proba(rating_data['X_dev'])[:, 1])

tr_auc, dev_auc

/Users/przivic/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:760: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.7050786145493632, 0.6466820465336702)

In [65]:
pipe = make_pipeline(
    W2VCrewFeatures(w2v, ['1st_star', '2nd_star', '3rd_star', 'director']),
    StandardScaler(),
    LogisticRegression()
)

In [66]:
pipe.fit(rating_data['X_train'], rating_data['y_train'] > 7.5)

tr_auc = roc_auc_score(rating_data['y_train'] > 7.5, pipe.predict_proba(rating_data['X_train'])[:, 1])
dev_auc = roc_auc_score(rating_data['y_dev'] > 7.5, pipe.predict_proba(rating_data['X_dev'])[:, 1])

tr_auc, dev_auc

/Users/przivic/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:760: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.7259572017485352, 0.6663948172895163)

In [67]:
pipe = make_pipeline(
    make_union(
        W2VCrewFeatures(w2v, ['1st_star', '2nd_star', '3rd_star']),
        W2VCrewFeatures(w2v, ['director'])
    ),
    StandardScaler(),
    LogisticRegression()
)

In [68]:
pipe.fit(rating_data['X_train'], rating_data['y_train'] > 7.5)

tr_auc = roc_auc_score(rating_data['y_train'] > 7.5, pipe.predict_proba(rating_data['X_train'])[:, 1])
dev_auc = roc_auc_score(rating_data['y_dev'] > 7.5, pipe.predict_proba(rating_data['X_dev'])[:, 1])

tr_auc, dev_auc

/Users/przivic/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:760: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.733849182294092, 0.6649772550701968)

In [72]:
pipe = make_pipeline(
    make_union(
        W2VCrewFeatures(w2v, ['1st_star', '2nd_star', '3rd_star']),
        make_pipeline(transformers.CrewFeatures('1st_star', min_cnt_movies=3), DictVectorizer(sparse=False)),
        make_pipeline(transformers.CrewFeatures('2nd_star', min_cnt_movies=3), DictVectorizer(sparse=False)),
        make_pipeline(transformers.CrewFeatures('3rd_star', min_cnt_movies=3), DictVectorizer(sparse=False)),
    ),
    GradientBoostingClassifier()
)

In [ ]:
pipe.fit(rating_data['X_train'], rating_data['y_train'] > 7.5)

tr_auc = roc_auc_score(rating_data['y_train'] > 7.5, pipe.predict_proba(rating_data['X_train'])[:, 1])
dev_auc = roc_auc_score(rating_data['y_dev'] > 7.5, pipe.predict_proba(rating_data['X_dev'])[:, 1])

tr_auc, dev_auc